In [1]:
import weave
from weave.ecosystem import keras as weave_keras

weave.use_frontend_devmode()

# Train Image Model

In [2]:
import wandb
import os
import random

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import train_test_split

import wandb
from wandb.keras import WandbCallback

In [4]:
saved_text_model = weave.get("local-artifact:///tmp/local-artifacts/KerasModel/6558c37f75b5b9d0a198d4a989ba9f3f")
saved_text_model

ERROR:ddtrace.internal.writer:failed to send traces to Datadog Agent at http://localhost:8126/v0.4/traces
Traceback (most recent call last):
  File "/Users/timothysweeney/.pyenv/versions/3.9.9/envs/weave_internal/lib/python3.9/site-packages/tenacity/__init__.py", line 407, in __call__
    result = fn(*args, **kwargs)
  File "/Users/timothysweeney/.pyenv/versions/3.9.9/envs/weave_internal/lib/python3.9/site-packages/ddtrace/internal/writer.py", line 446, in _send_payload
    response = self._put(payload, headers)
  File "/Users/timothysweeney/.pyenv/versions/3.9.9/envs/weave_internal/lib/python3.9/site-packages/ddtrace/internal/writer.py", line 398, in _put
    self._conn.request("PUT", self._endpoint, data, headers)
  File "/Users/timothysweeney/.pyenv/versions/3.9.9/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/http/client.py", line 1285, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Users/timothysweeney/.pyenv/versions/3.9.9/L

In [15]:
saved_text_model.val.predict(["Asd"]).tolist()

1/1 [==============================] - 0s 34ms/step


[[0.5092068314552307]]

In [17]:
weave_keras.call_string(saved_text_model, "single test string 223")

ERROR:ddtrace.internal.writer:failed to send traces to Datadog Agent at http://localhost:8126/v0.4/traces
Traceback (most recent call last):
  File "/Users/timothysweeney/.pyenv/versions/3.9.9/envs/weave_internal/lib/python3.9/site-packages/tenacity/__init__.py", line 407, in __call__
    result = fn(*args, **kwargs)
  File "/Users/timothysweeney/.pyenv/versions/3.9.9/envs/weave_internal/lib/python3.9/site-packages/ddtrace/internal/writer.py", line 446, in _send_payload
    response = self._put(payload, headers)
  File "/Users/timothysweeney/.pyenv/versions/3.9.9/envs/weave_internal/lib/python3.9/site-packages/ddtrace/internal/writer.py", line 398, in _put
    self._conn.request("PUT", self._endpoint, data, headers)
  File "/Users/timothysweeney/.pyenv/versions/3.9.9/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/http/client.py", line 1285, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Users/timothysweeney/.pyenv/versions/3.9.9/L

In [3]:
project = "weave_model_1"

default_config = {  # and include hyperparameters and metadata
        "learning_rate": 0.001,
        "epochs": 1,
        "batch_size": 64,
        "loss_function": "sparse_categorical_crossentropy",
        "architecture": "CNN",
        "dataset": "CIFAR-10",
        "train_count": 2500,
        "test_count": 200,
    }

CLASS_NAMES = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]


def Model():
    inputs = keras.layers.Input(shape=(32, 32, 3))

    x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(inputs)
    x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
    x = keras.layers.MaxPooling2D(pool_size=2)(x)

    x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
    x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)

    x = keras.layers.GlobalAveragePooling2D()(x)

    x = keras.layers.Dense(128, activation="relu")(x)
    x = keras.layers.Dense(32, activation="relu")(x)

    outputs = keras.layers.Dense(len(CLASS_NAMES), activation="softmax")(x)

    return keras.models.Model(inputs=inputs, outputs=outputs)

In [4]:
config = default_config
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train[::5] / 255.0, x_test[::5]  / 255.0
y_train, y_test = y_train[::5], y_test[::5]
(x_train, y_train) = (x_train[:config['train_count']], y_train[:config['train_count']])
(x_test, y_test) = (x_test[:config['test_count']], y_test[:config['test_count']])

In [5]:
image_model = Model()
optimizer = tf.keras.optimizers.Adam(config['learning_rate'])
image_model.compile(optimizer, config['loss_function'], metrics=["acc"])

_ = image_model.fit(
  x_train,
  y_train,
  epochs=config['epochs'],
  batch_size=config['batch_size'],
  validation_data=(x_test, y_test),
#   callbacks=[WandbCallback(
#       labels=CLASS_NAMES,
#       log_evaluation=True,
#   )],
)

2022-08-09 15:18:10.403565: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


40/40 [==============================] - 3s 50ms/step - loss: 2.2990 - acc: 0.1104 - val_loss: 2.2861 - val_acc: 0.1750


In [ ]:
published_image_model = weave.publish(image_model)
# published_image_model = weave.save(image_model)

In [ ]:
frog_small = "https://www.cs.toronto.edu/~kriz/cifar-10-sample/frog10.png"
ship_small = "https://www.cs.toronto.edu/~kriz/cifar-10-sample/ship10.png"
truck_small = "https://www.cs.toronto.edu/~kriz/cifar-10-sample/truck10.png"
plane_small = "https://www.cs.toronto.edu/~kriz/cifar-10-sample/airplane10.png"

frog_url = "https://www.canr.msu.edu/contentAsset/image/44bc1519-3b22-42b7-9d10-46e3f9f20309/fileAsset/filter/Resize,Jpeg/resize_w/750/jpeg_q/80"
ship_url = "https://www.itfseafarers.org/sites/default/files/styles/node_main/public/node/issues/image/Cruise%20Ship.jpg?itok=H5WzDqy3"
truck_url = "https://cdn.motor1.com/images/mgl/G3y6rA/s3/2022-ford-f-series-super-duty.jpg"
plane_url = "https://media.istockphoto.com/photos/passenger-airplane-flying-above-clouds-during-sunset-picture-id155439315?k=20&m=155439315&s=612x612&w=0&h=BvXCpRLaP5h1NnvyYI_2iRtSM0Xsz2jQhAmZ7nA7abA="

weave.ops.make_list(
    a=weave_keras.image_classification(published_image_model, frog_small),
    b=weave_keras.image_classification(published_image_model, ship_small),
    c=weave_keras.image_classification(published_image_model, truck_small),
    d=weave_keras.image_classification(published_image_model, plane_small),
    e=weave_keras.image_classification(published_image_model, frog_url),
    f=weave_keras.image_classification(published_image_model, ship_url),
    g=weave_keras.image_classification(published_image_model, truck_url),
    h=weave_keras.image_classification(published_image_model, plane_url)
)

# Train Language Model

From: https://keras.io/examples/nlp/text_classification_from_scratch/

In [4]:
import tensorflow as tf
import numpy as np

In [5]:
# !curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# !tar -xf aclImdb_v1.tar.gz
# !rm -r aclImdb/train/unsup

In [6]:
batch_size = 32
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
)
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
)
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

print(f"Number of batches in raw_train_ds: {raw_train_ds.cardinality()}")
print(f"Number of batches in raw_val_ds: {raw_val_ds.cardinality()}")
print(f"Number of batches in raw_test_ds: {raw_test_ds.cardinality()}")

from tensorflow.keras.layers import TextVectorization
import string
import re

# Having looked at our data above, we see that the raw text contains HTML break
# tags of the form '<br />'. These tags will not be removed by the default
# standardizer (which doesn't strip HTML). Because of this, we will need to
# create a custom standardization function.
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )


# Model constants.
max_features = 20000
embedding_dim = 128
sequence_length = 500

# Now that we have our custom standardization, we can instantiate our text
# vectorization layer. We are using this layer to normalize, split, and map
# strings to integers, so we set our 'output_mode' to 'int'.
# Note that we're using the default split function,
# and the custom standardization defined above.
# We also set an explicit maximum sequence length, since the CNNs later in our
# model won't support ragged sequences.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

vectorize_layer_non_custom = TextVectorization(
#     standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

# Now that the vocab layer has been created, call `adapt` on a text-only
# dataset to create the vocabulary. You don't have to batch, but for very large
# datasets this means you're not keeping spare copies of the dataset in memory.

# Let's make a text-only dataset (no labels):
text_ds = raw_train_ds.map(lambda x, y: x)
# Let's call `adapt`:
vectorize_layer.adapt(text_ds)
vectorize_layer_non_custom.adapt(text_ds)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


2022-08-09 15:54:33.852549: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
Number of batches in raw_train_ds: 625
Number of batches in raw_val_ds: 157
Number of batches in raw_test_ds: 782


In [7]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label


# Vectorize the data.
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

# Do async prefetching / buffering of the data for best performance on GPU.
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

In [8]:
from tensorflow.keras import layers

# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

text_model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
text_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [9]:
epochs = 1

# Fit the model using the train and test datasets.
text_model.fit(train_ds, validation_data=val_ds, epochs=epochs)

625/625 [==============================] - 46s 73ms/step - loss: 0.4861 - accuracy: 0.7276 - val_loss: 0.3090 - val_accuracy: 0.8706


In [10]:
# A string input
inputs = tf.keras.Input(shape=(1,), dtype="string")
# Turn strings into vocab indices
indices = vectorize_layer_non_custom(inputs)
# Turn vocab indices into predictions
outputs = text_model(indices)

# Our end to end model
end_to_end_model = tf.keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)

In [11]:
saved_text_model = weave.save(end_to_end_model)

INFO:tensorflow:Assets written to: /tmp/local-artifacts/KerasModel/working-X5ZTR0F1BV41/_obj/assets


INFO:tensorflow:Assets written to: /tmp/local-artifacts/KerasModel/working-X5ZTR0F1BV41/_obj/assets


In [12]:
saved_text_model

ERROR:ddtrace.internal.writer:failed to send traces to Datadog Agent at http://localhost:8126/v0.4/traces
Traceback (most recent call last):
  File "/Users/timothysweeney/.pyenv/versions/3.9.9/envs/weave_internal/lib/python3.9/site-packages/tenacity/__init__.py", line 407, in __call__
    result = fn(*args, **kwargs)
  File "/Users/timothysweeney/.pyenv/versions/3.9.9/envs/weave_internal/lib/python3.9/site-packages/ddtrace/internal/writer.py", line 446, in _send_payload
    response = self._put(payload, headers)
  File "/Users/timothysweeney/.pyenv/versions/3.9.9/envs/weave_internal/lib/python3.9/site-packages/ddtrace/internal/writer.py", line 398, in _put
    self._conn.request("PUT", self._endpoint, data, headers)
  File "/Users/timothysweeney/.pyenv/versions/3.9.9/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/http/client.py", line 1285, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Users/timothysweeney/.pyenv/versions/3.9.9/L

In [12]:
weave_keras.call_string(saved_text_model, "single test string")

TypeError: 'TypedDict' object is not subscriptable

In [ ]:
weave.panels.Table([
    {'input_str': "This is a test"},
    {'input_str': "This is a horrible test"},
    {'input_str': "I love weave"},
    {'input_str': "please work!"},
    {'input_str': "I am a person in a house"},
], columns=[
    lambda row: row['input_str'],
    lambda row: weave_keras.call_string_to_number(saved_text_model, row['input_str'])
])

In [ ]:
# published_text_model = weave.save(end_to_end_model)
published_text_model = weave.publish(end_to_end_model)

In [ ]:
weave_keras.call_string_to_number(published_text_model, "single test string")